## Importing libraries

In [ ]:
# importing the libraries
from torchsummary import summary
import pandas as pd
import numpy as np
from skimage.io import imread, imsave
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from scipy import ndimage

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


## Data
The dataset , I am working on is, emergency and non emergency vehicles images , which is available on kaggle.the datset has two files, one with 1646 images and the other .csv file with image name and the label.
0 here represents that the vehicle is non-emergency and 1 means it’s an emergency vehicle. 

In [ ]:
# loading dataset
data = pd.read_csv('/gdrive/MyDrive/emergency_vs_non-emergency_dataset/emergency_vs_non-emergency_dataset/emergency_train.csv')
data.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


## Loading the dataset
The dataset is loaded from the drive. The shape of training data shows, images are of the size 224*224 and channels are 3 that means images are coloured.


In [ ]:
# loading images
train_img = []
for img_name in tqdm(data['image_names']):
    image_path = '/gdrive/MyDrive/emergency_vs_non-emergency_dataset/emergency_vs_non-emergency_dataset/images/' + img_name
    img = imread(image_path)
    img = img/255
    train_img.append(img)

train_x = np.array(train_img)
train_y = data['emergency_or_not'].values
train_x.shape, train_y.shape

100%|██████████| 1646/1646 [10:18<00:00,  2.66it/s]


((1646, 224, 224, 3), (1646,))

## Train-test split
I have kept the test_size as 0.1 and hence 10% data will be randomly selected as the validation set and the remaining 90% will be used to train the model. We have 1,481 images in the training set which is quite less to train a deep learning model. I wanted to data augmentation for increasing the size of train data, but due to RAM limitation in colab, my session kept on crashing. So, had to drop that idea.

Now, only 1481 images in train dataset.


In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state = 13, stratify=train_y)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((1481, 224, 224, 3), (1481,)), ((165, 224, 224, 3), (165,)))

## Defining the CNN Model using Pytorch

In [ ]:
# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

### First,the images are converted to tensors.

In [ ]:
# converting training images into torch format
#train_x = train_x.reshape(7405, 3, 224, 224)
train_x  = torch.from_numpy(train_x)
train_x = train_x.float()

# converting the target into torch format
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y)


In [ ]:
print(train_x.shape) # train dataset shape

torch.Size([1481, 224, 224, 3])


In [ ]:
train_x = train_x.reshape(1481,3,224,224) # reshaping it before it is fed into the model with number of channels ahead of the image size

In [ ]:
print(train_x.shape)

torch.Size([1481, 3, 224, 224])


### similar process is repeated with test/val dataset

In [ ]:
# converting validation images into torch format
#val_x = val_x.reshape(165,224, 224,3)
val_x  = torch.from_numpy(val_x)
val_x = val_x.float()

# converting the target into torch format
val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)

In [ ]:
print(val_x.shape)

torch.Size([165, 224, 224, 3])


## Defining the architecture of CNN model
The architecture has 4 convolutional blocks and then 4 fully connected dense layers. The kernel size is 3, padding and stride  size are  1 for all the convolutional layers.After every convolutional layer, ReLu activation function is ised.
To avoid overfitting andrandomize the dataset, dropout of 0.25 is used after every convolutional layer and Linear layer.
For regularization and to make training more efficient,batch normalization layer is also used afterevery convolutional layer.


In [ ]:
torch.manual_seed(0)

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(32),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
            # Defining another 2D convolution layer
            Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(64),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
            # Defining another 2D convolution layer
            Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(128),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
            # Defining another 2D convolution layer
            Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(128),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
        )

        self.linear_layers = Sequential(
            Linear(128 * 14 * 14, 512),
            ReLU(inplace=True),
            Dropout(),
            Linear(512, 256),
            ReLU(inplace=True),
            Dropout(),
            Linear(256,10),
            ReLU(inplace=True),
            Dropout(),
            Linear(10,2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.000075)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.25, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.25, inplace=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Dropout(p=0.25, inplace=False)
    (15): Con

In [ ]:
torch.manual_seed(0)

# batch size of the model
batch_size = 64

# number of epochs to train the model
n_epochs = 20

for epoch in range(1, n_epochs+1):

    train_loss = 0.0
        
    permutation = torch.randperm(train_x.size()[0])

    training_loss = []
    for i in tqdm(range(0,train_x.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

100%|██████████| 24/24 [00:07<00:00,  3.34it/s]


epoch: 	 1 	 training loss: 	 0.6822157576680183


100%|██████████| 24/24 [00:06<00:00,  3.47it/s]


epoch: 	 2 	 training loss: 	 0.6759209657708803


100%|██████████| 24/24 [00:06<00:00,  3.47it/s]


...

epoch: 	 18 	 training loss: 	 0.4779381876190503


100%|██████████| 24/24 [00:06<00:00,  3.50it/s]


epoch: 	 19 	 training loss: 	 0.4670604070027669


100%|██████████| 24/24 [00:06<00:00,  3.50it/s]

epoch: 	 20 	 training loss: 	 0.428358007222414


In [ ]:
torch.manual_seed(0)
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in tqdm(range(0,train_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices],train_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x).cuda()

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i].cpu(),prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))

100%|██████████| 24/24 [00:02<00:00,  9.06it/s]

training accuracy: 	 0.8064959490740741


In [ ]:
print(val_x.shape)

torch.Size([165, 224, 224, 3])


In [ ]:
val_x=val_x.reshape(165,3,224,224)

In [ ]:
# checking the performance on validation set
torch.manual_seed(0)
output = model(val_x.cuda())
softmax = torch.exp(output).cpu()
prob = list(softmax.detach().numpy())
predictions = np.argmax(prob, axis=1)
accuracy_score(val_y, predictions)

0.703030303030303